# Recommendation System - Model 1

## Import packages and dataframes

In [1]:
import numpy as np
import pandas as pd
import random as rd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

rd.seed(123)

In [350]:
# import all the files

donationsDF = pd.read_csv('Donations.csv')
donorsDF = pd.read_csv('Donors.csv')
projectsDF = pd.read_csv('Projects.csv')
schoolsDF = pd.read_csv('Schools.csv')
resourcesDF = pd.read_csv('Resources.csv')

/Users/prastha/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Cleaning: Remove one-time donors

In [364]:
donorsDF_merged = donorsDF.merge(donationsDF, on = 'Donor ID', how = 'inner') # donation ID is the primary key here

In [417]:
# Get the count of donations a donor has made
df_temp1 = donorsDF_merged.groupby(['Donor ID']).size().reset_index(name='Num Donations')
df_temp2 = df_temp1[df_temp1['Num Donations'] > 4] # removing all the donors who have donated less than 5 times

print("We got rid of %d donors." % (df_temp1.shape[0] - df_temp2.shape[0]))
print("The shape of the remaining dataframe:", df_temp2.shape)

We got rid of 1889912 donors.
The shape of the remaining dataframe: (129050, 2)


## Recurring donors for modeling

In [418]:
uniqDonorIDList = list(df_temp2['Donor ID']) # list of recurring donors
df_final = donorsDF_merged[donorsDF_merged['Donor ID'].isin(uniqDonorIDList)] # recurring donors

del df_temp1, df_temp2, donorsDF_merged
df_final.shape

(2169542, 11)

In [6]:
def removeDuplicates(df, col):
    df = df.drop_duplicates(subset = [col]) # removes duplicates from the column `col`
    return df

In [7]:
df_final = removeDuplicates(df_final, 'Donation ID') # removes duplicate Donation IDs

# donation ID is the primary key here
df_final = df_final.merge(projectsDF, on = "Project ID", how = "inner") # adding projects to the final dataframe

df_final = df_final.merge(schoolsDF, on = "School ID", how = "inner") # adding schools to the final dataframe

In [8]:
# discretizing (binning) the project cost

projCost = pd.DataFrame((df_final['Project Cost'] // 500 + 1) * 500).astype('int')

projCost[projCost['Project Cost'] > 3000] = 3000

projCost['Project Cost'] = projCost['Project Cost'].astype('str')
projCost[projCost['Project Cost'] == '3000'] = "3000+"

df_final['Project Cost'] = projCost
df_final = df_final.rename(columns = {"Project Cost": "Project Cost (Up to)"})

print(df_final.shape)

(2126224, 36)


In [9]:
#we are only considering all the donations that came to projects in California

df_final = removeDuplicates(df_final, 'Donation ID') # removes duplicate Donation IDs
print("The original dataframe size: ", df_final.shape)

df_final_cal = df_final[df_final['School State'] == 'California']
print("The updated dataframe size including only California: ", df_final_cal.shape)

df_final_cal.sample(2)

The original dataframe size:  (2126222, 36)
The updated dataframe size including only California:  (321259, 36)


,Donor ID,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,...,Project Current Status,Project Fully Funded Date,School Name,School Metro Type,School Percentage Free Lunch,School State,School Zip,School City,School County,School District
157400,f37b9ff133242d08658ca2b17b3ab84c,Dillon,South Carolina,No,295,4215fe736e726455d5b78b2873557742,d7b5c9c526b7588a15c517604f050506,No,5.0,2595,...,Fully Funded,2016-05-10,F D Roosevelt Elementary School,suburban,90.0,California,90260,Lawndale,Los Angeles,Lawndale Elem School District
409383,e7692ca3a0fe35653c173de80adfc880,Princeton Junction,New Jersey,No,085,02f2f8fc72b4714f5c6b98c04edb5913,59fb2b37ee8762945df9252b54aaf6be,No,74.0,1612,...,Fully Funded,2014-11-22,Newark Memorial High School,suburban,56.0,California,94560,Newark,Alameda,Newark Unified School District


In [10]:
del df_final

## Experiment Lab

In [221]:
features = ['Donor ID', 'Project ID', 'Donation Received Date']
df_donor = df_final_cal[features]
donor_list = set(df_donor['Donor ID'])
df_donor = df_donor.drop_duplicates(subset = ['Donor ID', 'Project ID'])
df_donor = df_donor.sort_values(by = ['Donor ID','Donation Received Date'])

df_donor['Donation Received Date'] = pd.to_datetime(df_donor['Donation Received Date'])

In [222]:
df_donor.sample(5)

,Donor ID,Project ID,Donation Received Date
1703748,59b9c7a901b3d0a9c319b5f40a8208bd,b1835b789b653e72177e25cdf590b6c6,2013-08-28 01:34:25
297738,0c8f0a3bcc4355408a65c2e048e7a08e,5861298ad07837b1e38a0c6f8e05c165,2017-08-31 13:24:57
778606,55aa4bb22cda065e8658acb371c10135,2a90ff6e0417f1d6dc7f85fd853afce9,2017-05-05 01:10:41
253449,7705e9d9b7f95f7e768ac5501b64d2aa,e35c1ee34e7363f421ddbe087fc52b90,2017-02-27 18:21:38
958563,9b3831f3a7086e17220a646756b3f668,4c42367829ec721eb87d58e0970507a5,2015-08-18 15:35:10


In [298]:
def trainTestSplit(df_donor, donor_id): # one donor_id at a time
    donated_projects = list(df_donor[df_donor['Donor ID'] == donor_id]['Project ID'])
    
    
    train = 0
    train = int(round(0.8 * len(donated_projects), 0))
    train_projects = donated_projects[:train]
    test_projects = donated_projects[train:]
    
    last_donation_date = df_donor[(df_donor['Donor ID'] == donor_id) & (df_donor['Project ID'] == train_projects[-1])]['Donation Received Date']
    last_donation_date = pd.Timestamp(str(last_donation_date).split()[1])
    
    
    return train_projects, test_projects, last_donation_date

# test the function here
train_projects, test_projects, last_donation_date = trainTestSplit(df_donor, '6464c0ad878eca6368c913feda1c52c4')

In [293]:
# because we are doing content-based filtering, here are the features of the projects we are interested in:

ohFeatures = ['Project ID', 'Project Expiration Date', 'Project Cost (Up to)', 'Project Type','Project Subject Category Tree',
       'Project Grade Level Category', 'Project Resource Category',
       'Project Current Status',
       'School Metro Type', 'School State']

# features we will do one hot encoding on
dropCol = ['Project Cost (Up to)', 'Project Type', 'Project Subject Category Tree', 'Project Grade Level Category',
           'Project Resource Category', 'Project Current Status',
           'School Metro Type', 'School State']

def oheDataFrame(dataframe): # the dataframe could either be training or validation set
    projFeatures = dataframe[ohFeatures] # only considering the selected features (ohFeatures)
    projFeatures = removeDuplicates(projFeatures, 'Project ID')
    projFeatures = projFeatures.dropna() # drop the null values as well
    projFeatures = projFeatures.sort_values(by = ['Project ID'])

    featuresList = list(projFeatures.columns)
    
    enc = OHE(handle_unknown = 'ignore')
    enc.fit(projFeatures.iloc[:,2:]) # fit the one hot vector on our dataframe
    
    # convert the one-hot matrix into a dataframe
    oneHotMatrix = pd.DataFrame(enc.transform(projFeatures.iloc[:,2:]).toarray())
    projOHM = pd.concat([projFeatures.reset_index(drop = True), oneHotMatrix.reset_index(drop = True)], axis = 1)
    projOHM = projOHM.drop(dropCol, axis = 1) # dropping the categorical columns because they have already been vectorized

    # data dictionary to track the one hot matrix

    dataDict = list(enc.get_feature_names(dropCol)) # please note that any new input should strictly should the indexes
                                                    # mentioned in data dictionary

    
    # scale and normalize the dataset

    ohmDF = pd.DataFrame(StandardScaler().fit_transform(projOHM.iloc[:,2:]))
    ohmDFwProj = pd.concat([projOHM[['Project ID', 'Project Expiration Date']], ohmDF], axis = 1)
    ohmDFwProj['Project Expiration Date'] =  pd.to_datetime(ohmDFwProj['Project Expiration Date'])
    ohmDFwProj = ohmDFwProj.sort_values(by = ['Project Expiration Date'])
    
    return ohmDFwProj

ohmDFwProj = oheDataFrame(df_final_cal)

In [225]:
ohmDFwProj.sample(5)

,Project ID,Project Expiration Date,0,1,2,3,4,5,6,7,...,80,81,82,83,84,85,86,87,88,89
83173,daefd8bb3daf9045be45318cce63225e,2013-09-21,-0.719591,-0.32398,-0.209574,-0.147667,-0.175159,1.060486,-0.109236,-0.096734,...,-0.057115,-0.378367,0.401088,-0.116301,-0.14204,1.342352,-0.162189,-0.205339,-1.121844,0.0
10605,1b792072713faceb98b1007d37d7a7df,2017-11-22,1.389678,-0.32398,-0.209574,-0.147667,-0.175159,-0.942964,-0.109236,-0.096734,...,-0.057115,-0.378367,0.401088,-0.116301,-0.14204,-0.744961,-0.162189,-0.205339,0.891389,0.0
58633,9a92c32e4f9f0fc192631067ec8c14f2,2013-09-08,1.389678,-0.32398,-0.209574,-0.147667,-0.175159,-0.942964,-0.109236,-0.096734,...,-0.057115,-0.378367,0.401088,-0.116301,-0.14204,-0.744961,-0.162189,-0.205339,0.891389,0.0
64178,a94f2cf5b6660afbed0034ad26ed58b4,2018-08-27,-0.719591,-0.32398,-0.209574,-0.147667,-0.175159,1.060486,-0.109236,-0.096734,...,-0.057115,-0.378367,0.401088,-0.116301,-0.14204,1.342352,-0.162189,-0.205339,-1.121844,0.0
90790,ef489c9d47fab94c25aab0770e501d74,2015-06-17,1.389678,-0.32398,-0.209574,-0.147667,-0.175159,-0.942964,-0.109236,-0.096734,...,-0.057115,2.642939,-2.493219,-0.116301,-0.14204,1.342352,-0.162189,-0.205339,-1.121844,0.0


In [226]:
# this function takes a list of donated projects and averages their feature to compile a donation profile

def donationProfile(ohe_df, train_projects): #train_projects is the list of donated projects
                                                #ohe_df is the one-hot encoded df of all projects
        
    projs = ohe_df[ohe_df['Project ID'].isin(train_projects)]
    projs_avg = pd.DataFrame(projs.iloc[:,2:].mean(axis = 0)).T
    
    return projs_avg

projs_avg = donationProfile(ohmDFwProj, train_projects)

In [286]:
def cosineSimilarity(ohe_df, project, train_projects, last_donation, numRec): # only works for one observation (project) at a time
                                                # project here is the average project
        
    
    ohe_df = ohe_df[-ohe_df['Project ID'].isin(train_projects)]
    
    
    # getting the cosine similarity between our feature matrix and test sample
    cosSim = cosine_similarity(ohe_df.iloc[:,2:], project) # one-hot encoded dataframe is the dataframe here
    recProjects = pd.concat([ohe_df.iloc[:,:2], pd.DataFrame(cosSim)], axis = 1)

    recProjects = recProjects[recProjects['Project Expiration Date'] > last_donation] # only recommend projects that expire after the donation date
    
    recProjects = recProjects.sort_values(by = 0, ascending = False) # sorting the similarity in descending order
    
    topRec = list(recProjects.head(numRec).iloc[:,0]) # the first column is the project ID
    
    return topRec

In [256]:
def getTopPrecision(df_donor, donor_id, numRec, ohe_df = ohmDFwProj):
    train_projects, test_projects, last_donation_date = trainTestSplit(df_donor, donor_id)
    
    projs_avg = donationProfile(ohe_df, train_projects)
    
    top10Rec = cosineSimilarity(ohe_df, projs_avg, train_projects, last_donation_date, numRec)
    
    hits = 0
    
    for i in top10Rec:
        if i in test_projects:
            hits += 1
    
    return hits

# getTopPrecision(df_donor, '0002806acba3480d65cec587c1afb1ee', 10, ohmDFwProj)

In [301]:
# rec_precision_results = getPrecision(donor_list, df_donor, ohmDFwProj)

print(rec_precision_results)

{5: 0.0, 10: 0.0, 25: 8e-05, 50: 8e-05, 100: 8e-05}


## Recommendation System Execution

In [299]:
num_recs = [5, 10, 25, 50, 100] # number of recommendations
training_datapoints = 1000 # the number of donors we wanna look at
hits = 0

precision_results = {}

for i in num_recs:
    for j in rd.sample(donor_list, training_datapoints):
        hits += getTopPrecision(df_donor, j, i, ohe_df)

    precision = hits / (i * training_datapoints)
    precision_results[i] = precision

In [302]:
# rec_precision_results = getPrecision(donor_list, df_donor, ohmDFwProj)

print(rec_precision_results)

{5: 0.0, 10: 0.0, 25: 8e-05, 50: 8e-05, 100: 8e-05}


In [332]:
rec_results = pd.DataFrame.from_dict(rec_precision_results, orient = 'index').reset_index()
rec_results[0] = rec_results[0] * 100
rec_results.columns = ['# of Recommendations', 'Recommendation System Precision']

In [333]:
randomrec_results = pd.DataFrame({5: [0], 10: [0], 25: [0], 50: [0], 100: [0]})

randomrec_results = randomrec_results.T.reset_index()
randomrec_results.columns = ['# of Recommendations', 'Random Recommendation Precision']

In [334]:
rec_results_final = randomrec_results.merge(rec_results, on = '# of Recommendations')
rec_results_final

,# of Recommendations,Random Recommendation Precision,Recommendation System Precision
0,5,0,0.000
1,10,0,0.000
2,25,0,0.008
3,50,0,0.008
4,100,0,0.008


In [335]:
df_final_cal.head(1)

,Donor ID,Donation Received Date,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,Donation Amount,...,Project Current Status,Project Fully Funded Date,School Name,School Metro Type,School Percentage Free Lunch,School State,School Zip,School City,School County,School District
8,00002d44003ed46b066607c5455a999a,2016-10-25 20:15:11,Winton,California,Yes,953,e2beb818569f66adaa4ced21ca299ac6,08ed72ce14c548e8131a7dd7b8561988,Yes,10.0,...,Fully Funded,2016-12-26,Herbert Hoover Middle School,urban,82.0,California,95340,Merced,Merced,Merced City School District


In [341]:
f = ['Donor ID', 'Project ID', 'Donation Received Date']

t = df_final_cal[f]
t = removeDuplicates(t, 'Project ID')
t[t['Donor ID'] == '00002d44003ed46b066607c5455a999a']

,Donor ID,Project ID,Donation Received Date
8,00002d44003ed46b066607c5455a999a,e2beb818569f66adaa4ced21ca299ac6,2016-10-25 20:15:11
106,00002d44003ed46b066607c5455a999a,64f54f1efcbeb986114a7a13e6b27257,2017-01-16 14:20:10
3,00002d44003ed46b066607c5455a999a,e09933470f4256cc2643341c1d299e55,2017-02-01 18:53:25
1,00002d44003ed46b066607c5455a999a,2f53e5f31890e647048ac217cda3b83f,2017-04-01 01:08:50
112,00002d44003ed46b066607c5455a999a,c5821d32012efd7df4f6fa12e230e991,2017-10-18 14:34:11


In [348]:
project_lst = list(t['Project ID'])

rand = pd.DataFrame(rd.sample(project_lst, 5))
rand.columns = ['Recommended Projects']

rand

,Recommended Projects
0,fcbaf20dd4acc6db01b41d57048b988d
1,48314c7720ad2402d0ede9b3b4bcfce2
2,cc0da336e6dbe04f8e7a24076d37bee4
3,812b74963373d4923412eab4667a18e4
4,d6eaf1127585aac68dedb15e5700bfff


In [355]:
ohmDFwProj.shape

(97187, 92)

In [358]:
len(np.unique(df_final_cal['Donor ID']))

34902

In [361]:
def trainTestSplit(df_donor, donor_id): # one donor_id at a time
    donated_projects = list(df_donor[df_donor['Donor ID'] == donor_id]['Project ID'])
    
    
    train = 0
    train = int(round(0.8 * len(donated_projects), 0))
    train_projects = donated_projects[:train]
    test_projects = donated_projects[train:]
    
    last_donation_date = df_donor[(df_donor['Donor ID'] == donor_id) & (df_donor['Project ID'] == train_projects[-1])]['Donation Received Date']
    last_donation_date = pd.Timestamp(str(last_donation_date).split()[1])
    
    
    return donated_projects, train_projects, test_projects, last_donation_date

# test the function here
donated_projects, train_projects, test_projects, last_donation_date = trainTestSplit(df_donor, '00002d44003ed46b066607c5455a999a')

In [366]:
np.mean(projectsDF['Project Cost'])

741.5240377669335

In [367]:
projectsDF.columns

Index(['Project ID', 'School ID', 'Teacher ID',
       'Teacher Project Posted Sequence', 'Project Type', 'Project Title',
       'Project Essay', 'Project Short Description', 'Project Need Statement',
       'Project Subject Category Tree', 'Project Subject Subcategory Tree',
       'Project Grade Level Category', 'Project Resource Category',
       'Project Cost', 'Project Posted Date', 'Project Expiration Date',
       'Project Current Status', 'Project Fully Funded Date'],
      dtype='object')

In [368]:
schoolsDF.columns

Index(['School ID', 'School Name', 'School Metro Type',
       'School Percentage Free Lunch', 'School State', 'School Zip',
       'School City', 'School County', 'School District'],
      dtype='object')

In [384]:
ab = pd.merge(projectsDF, schoolsDF, on = 'School ID', how = 'left')

In [385]:
len(ab[ab['School State'] == 'California'])

167107

In [374]:
len(schoolsDF[schoolsDF['School State'] == 'California'])

8457

In [377]:
resourcesDF.columns

Index(['Project ID', 'Resource Item Name', 'Resource Quantity',
       'Resource Unit Price', 'Resource Vendor Name'],
      dtype='object')

In [386]:
ab = ab.merge(resourcesDF, on = 'Project ID')

In [387]:
len(ab[ab['School State'] == 'California'])

999796

In [389]:
cd = removeDuplicates(ab, 'School ID')

In [390]:
len(cd[cd['School State'] == 'California'])

8417

In [391]:
len(resourcesDF)

7210448

In [392]:
df_final_cal.columns

Index(['Donor ID', 'Donation Received Date', 'Donor City', 'Donor State',
       'Donor Is Teacher', 'Donor Zip', 'Project ID', 'Donation ID',
       'Donation Included Optional Donation', 'Donation Amount',
       'Donor Cart Sequence', 'School ID', 'Teacher ID',
       'Teacher Project Posted Sequence', 'Project Type', 'Project Title',
       'Project Essay', 'Project Short Description', 'Project Need Statement',
       'Project Subject Category Tree', 'Project Subject Subcategory Tree',
       'Project Grade Level Category', 'Project Resource Category',
       'Project Cost (Up to)', 'Project Posted Date',
       'Project Expiration Date', 'Project Current Status',
       'Project Fully Funded Date', 'School Name', 'School Metro Type',
       'School Percentage Free Lunch', 'School State', 'School Zip',
       'School City', 'School County', 'School District'],
      dtype='object')

In [394]:
len(df_final_cal[df_final_cal['Donor State'] == 'California'])

230798

In [397]:
teachersDF = pd.read_csv('Teachers.csv')

teachersDF.columns

Index(['Teacher ID', 'Teacher Prefix', 'Teacher First Project Posted Date'], dtype='object')

In [405]:
teachersDF.shape

(402900, 3)

In [414]:
len(t[t['School State'] == 'California'])

97189

In [412]:
t = pd.merge(df_final_cal, resourcesDF, on = 'Project ID')

t = removeDuplicates(df_final_cal, 'Project ID')



In [416]:
df_final

NameError: name 'df_final' is not defined